# Tweet topic classification

In [40]:
import re
import nltk
import glob
import random
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [41]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\petro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\petro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Read and preprocess data. Distribute data sets.

In [43]:
filenames = glob.glob("data/*.txt")
print(filenames)

['data\\business.txt', 'data\\entertainment.txt', 'data\\health.txt', 'data\\politics.txt', 'data\\sports.txt', 'data\\technology.txt']


In [44]:
labled_tweets = []
all_words = []
stpwords = stopwords.words('english')

for filename in filenames:
    file = open(filename, encoding='utf-8').read()
    for tweet in file.split('\n'):
        tweet = re.sub(r'[^\w\s]', '', tweet)
        tweet = re.sub(" \d+", " ", tweet)
        tweet = [i.lower() for i in list(set(word_tokenize(tweet)) - set(stpwords))]
        all_words += tweet
        labled_tweets.append((tweet, filename[5:-4]))  # extract target names from filename

random.shuffle(labled_tweets)
print(labled_tweets[0:5])

[(['scenes', 'insight', 'behind', 'committee', 'referees', 'preparation', 'referee', 'chariman', 'fifa', 'pierluigi', 'gives', 'collina'], 'sports'), (['shahrukh', 'aamir', 'amrish', 'ever', 'khan', 'mimicry', 'part', 'best', 'ampamp', 'more', 'puri', 'kajol', 'salman'], 'entertainment'), (['strike', 'obama', 'launch', 'wanted', 'syria', 'majority', 'voted', 'republicans'], 'politics'), (['politics', 'no', 'allowed', 'in', 'now', 'rhealth'], 'health'), (['western', 'drive', 'crammed', 'hard', 'tb', 'digital', 'storage', 'heliumfilled'], 'technology')]


In [46]:
word_features = list(all_words)
print(word_features[0:5])

['studying', 'report', 'financial', 'year', 'govt']


In [47]:
def get_features(tweet_text):
    tweet_words = set(tweet_text)
    features = {}
    for word in word_features:
        features['contains(%s)' % str(word)] = (word in tweet_words)
    return features

In [48]:
feature_set = [(get_features(text), label) for (text, label) in labled_tweets]

In [49]:
n = 400
train_set = feature_set[n:]
test_set = feature_set[:n]
print(len(train_set), len(test_set))

2733 400


In [50]:
print(test_set[0])

({'contains(studying)': False, 'contains(report)': False, 'contains(financial)': False, 'contains(year)': False, 'contains(govt)': False, 'contains(changing)': False, 'contains(rs)': False, 'contains(private)': False, 'contains(manage)': False, 'contains(deal)': False, 'contains(property)': False, 'contains(crore)': False, 'contains(lenders)': False, 'contains(bankers)': False, 'contains(luring)': False, 'contains(plus)': False, 'contains(salaries)': False, 'contains(cos)': False, 'contains(luxury)': False, 'contains(wealth)': False, 'contains(cars)': False, 'contains(management)': False, 'contains(reason)': False, 'contains(new)': False, 'contains(l)': False, 'contains(boss)': False, 'contains(friday)': False, 'contains(competition)': False, 'contains(said)': False, 'contains(commission)': False, 'contains(concerns)': False, 'contains(transaction)': False, 'contains(europe)': False, 'contains(would)': False, 'contains(raise)': False, 'contains(european)': False, 'contains(billion)': F

Train and test

In [55]:
NaiveBayes_classifier = nltk.NaiveBayesClassifier.train(train_set)
print("NaiveBayes accuracy:", (nltk.classify.accuracy(NaiveBayes_classifier, test_set)) * 100)

NaiveBayes accuracy: 74.25


In [51]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(train_set)
print("LinearSVC_classifier accuracy:", (nltk.classify.accuracy(LinearSVC_classifier, test_set)) * 100)

LinearSVC_classifier accuracy: 76.75


In [52]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression(multi_class='ovr'))  # one-vs-rest
LogisticRegression_classifier.train(train_set)
print("LogisticRegression_classifier accuracy:", (nltk.classify.accuracy(LogisticRegression_classifier, test_set)) * 100)

LogisticRegression_classifier accuracy: 75.75


In [53]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(train_set)
print("MNB_classifier accuracy:", (nltk.classify.accuracy(MNB_classifier, test_set)) * 100)

MNB_classifier accuracy: 78.5


In [54]:
SGD_classifier = SklearnClassifier(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))
SGD_classifier.train(train_set)
print("SGD_classifier accuracy:", (nltk.classify.accuracy(SGD_classifier, test_set)) * 100)

SGD_classifier accuracy: 76.0


Save

In [56]:
save_word_features = open("word_features.pickle", "wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

save_classifier = open("NaiveBayes.pickle", "wb")
pickle.dump(NaiveBayes_classifier, save_classifier)
save_classifier.close()

save_classifier = open("SVC.pickle", "wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

save_classifier = open("LogisticRegression.pickle", "wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

save_classifier = open("MNB.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

save_classifier = open("SGD.pickle","wb")
pickle.dump(SGD_classifier, save_classifier)
save_classifier.close()